In [39]:
# nuclio: ignore
import nuclio

## Define environment variables

In [40]:
# nuclio: ignore
from os import environ
version = '1.0'
#environ['MODEL_FILE'] = 'saved_model.pb'.format(version)
environ['MODEL_FILE'] = 'saved_model.pb'

In [41]:
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c MODEL_FILE=${MODEL_FILE}
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=/User/docs/site/en/tutorials/images/saved_models/1551857232/${MODEL_FILE}

%nuclio: setting 'MODEL_FILEPATH' environment variable


In [42]:
%%nuclio cmd -c
pip install --force tensorflow
pip install --force tensorflow_hub
pip install v3io_frames
pip install dask_ml
apt-get update && apt-get install -y wget
mkdir -p /tmp/mlmodel

In [43]:
%nuclio cmd -c wget -O /tmp/mlmodel/${MODEL_FILE} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://10.96.108.8:8081/users/${V3IO_USERNAME}/demos/netops/models/${MODEL_FILE}

## Set config

In [44]:
%%nuclio config 
spec.triggers.secs.attributes.interval = "60s"
spec.build.baseImage = "python:3.6-jessie"
spec.volumes.volume.name = "fs"
spec.volumes.volume.flexVolume.driver = "v3io/fuse"
spec.volumes.volume.flexVolume.secretRef.name = "jupyter-63mzkiyfdm-pgruh-v3io-fuse"
spec.volumes.volumeMount.volumeMount.name = "fs"
spec.volumes.volumeMount.volumeMount.mountPath = "/v3io"
spec.volumes.volumeMount.volumeMount.readOnly = False

%nuclio: setting spec.triggers.secs.attributes.interval to '60s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'
%nuclio: setting spec.volumes.volume.name to 'fs'
%nuclio: setting spec.volumes.volume.flexVolume.driver to 'v3io/fuse'
%nuclio: setting spec.volumes.volume.flexVolume.secretRef.name to 'jupyter-63mzkiyfdm-pgruh-v3io-fuse'
%nuclio: setting spec.volumes.volumeMount.volumeMount.name to 'fs'
%nuclio: setting spec.volumes.volumeMount.volumeMount.mountPath to '/v3io'
%nuclio: setting spec.volumes.volumeMount.volumeMount.readOnly to False


## Define function init

In [45]:
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
import v3io_frames as v3f
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define v3io TSDB client
client = v3f.Client('framesd:8081')



# Define parameters

In [46]:
image_size = 160 # All images will be resized to 160x160
IMG_SHAPE = (image_size, image_size, 3)
batch_size = 32
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   fname="cats_and_dogs_filtered.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)
validation_dir = os.path.join(base_dir, 'validation')


In [89]:
print (base_dir)
print (validation_dir)

/User/.keras/datasets/cats_and_dogs_filtered
/User/.keras/datasets/cats_and_dogs_filtered/validation


In [ ]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.trainable = True
new_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
new_model.summary()

In [ ]:
%%nuclio handler
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
prediction = datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

for item in prediction.classes:
    print('Image is a ' + {False: 'Cat', True :'Dog'}[item])
    
    
    

In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
#print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
#print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

************

In [ ]:



validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

for item in validation_generator.classes:
    print('Image is a ' + {False: 'Cat', True :'Dog'}[item])

In [ ]:
%nuclio deploy -n catsanddogs -p test1 -v

In [ ]:
# nuclio: ignore
nuclio.magic.print_handler_code()